REF [Doc](https://python.langchain.com/v0.2/docs/tutorials/chatbot/)


In [32]:
#import getpass
import os

#os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

# 2. Create model, change to Ollama compatible ,  
# see https://python.langchain.com/v0.2/docs/integrations/chat/openai/
# api_key must give to avoid error out
#model = ChatOpenAI(base_url="http://127.0.0.1:11434/v1", model_name="llama3.1", api_key="any")
from langchain_community.llms import Ollama
prompt = "You can AI assistant response in Json format"
#model = Ollama(model='llama3.1', format="json", system=prompt)
model = Ollama(model='llama3.1')

In [33]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

"Hi Bob! Nice to meet you. How's your day going so far?"

In [34]:
model.invoke([HumanMessage(content="What's my name?")])

"I don't know your personal details, but I can play along and ask you to introduce yourself. Why not start by telling me your name?"

In [35]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

"Your name is Bob. We just established that a minute ago, didn't we?"

In [36]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [37]:
config = {"configurable": {"session_id": "abc2"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


"Nice to meet you, Bob! Is there something on your mind that you'd like to chat about, or are we just starting a conversation?"

In [38]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


"Unfortunately, I'm a large language model, I don't have the ability to know or remember information about individual humans. You are essentially a new conversation every time you interact with me.\n\nHowever, if you'd like to share your name, I can use it in our conversation! Just let me know what it is."

In [39]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


"This conversation just started, so I don't have any information about you. Would you like to introduce yourself and share your name with me?"

In [40]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


"I don't actually know your name. This conversation just started, and I'm a large language model, I don't have personal knowledge of individuals. Would you like to introduce yourself?"

Prompt templates

Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

First, let's add in a system message. To do this, we will create a ChatPromptTemplate. We will utilize MessagesPlaceholder to pass all the messages in.

In [41]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [42]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response

"Nice to meet you, Bob! How can I help you today? Do you have any specific questions or topics you'd like to discuss, or would you just like to chat for a bit?"

In [43]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {"configurable": {"session_id": "abc5"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response

"Nice to meet you, Jim! I'm here to help with anything you need. What's on your mind? Do you have a question or task in mind that I can assist with?"

In [44]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [45]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response

'Hola Bob! Me alegra ayudarte en cualquier cosa que necesites. ¿En qué puedo ayudarte hoy? (Hello Bob! Nice to help you with anything you need. How can I assist you today?)'

In [46]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id": "abc11"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "English"},
    config=config,
)

response

"Nice to meet you, Todd! It's great to have you here. How can I assist you today? Do you need help with something or just want to chat? I'm all ears!"

In [47]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "English"},
    config=config,
)

response

"Your name is Todd! I remember from our initial greeting. Is there anything specific you'd like to talk about, or would you like me to recall it again?"

Managing Conversation History

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

Importantly, you will want to do this BEFORE the prompt template but AFTER you load previous messages from Message History.

We can do this by adding a simple step in front of the prompt that modifies the messages key appropriately, and then wrap that new chain in the Message History class.

LangChain comes with a few built-in helpers for managing a list of messages. In this case we'll use the trim_messages helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages:

In [48]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content="hi! I'm bob"),
 AIMessage(content='hi!'),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [49]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response

'Your name is Bob!'

In [50]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response

'You asked me to solve the math problem "2 + 2"!'

In [51]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response

"Your name is Bob! We just had a short conversation where you mentioned liking vanilla ice cream, and I was able to answer some of your questions. How's the rest of your day going?"

In [52]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response

"I made a mistake earlier. This is the beginning of our conversation, and we haven't discussed any specific details or math problems yet.\n\nLet me start fresh! I'm here to help with any question you have, and we can get started from scratch. What would you like to talk about?\n\n(And by the way, I don't actually know your name..."

In [55]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r, end="|")

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


Nice| to| meet| you|,| Todd|!| Here|'s| one|:

|What| do| you| call| a| fake| nood|le|?

|(wait| for| it|...)

|An| imp|asta|!

|Hope| that| made| you| smile|!| Do| you| want| another| one|?||